#WeatherPy

In [1]:
# Setup and Importing Dependencies
import datetime
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import requests
import scipy.stats as st
from scipy.stats import linregress
import time

In [2]:
# Imports and defines API Key
from api_keys import weather_api_key
api_key = weather_api_key

##Generate Cities List

In [3]:
# Creating list to be filled with cities
from citipy import citipy
citylist = []

# Gathers random list of coordinates(Lat,Lng) and adds it to list
randomlatlist= np.random.uniform(-90,90,1600)
randomlonglist= np.random.uniform(-180,180,1600)
coordlist = [(randomlatlist[i], randomlonglist[i]) for i in range(0, len(randomlatlist))]

# Takes a look at the latitude and longitude and finds the nearest city
for coord in coordlist:
    city = citipy.nearest_city(coord[0], coord[1]).city_name
    
    #If this city is a unique city not yet in the citylist, this adds it to the city list
    if city not in citylist:
        citylist.append(city)

# Prints number of cities. This is to make sure I've collected at least 500 cities to create data from.
print(len(citylist))

634


###Perform API Calls
* Performs a weather check on each city using a series of successive API calls.
* Includes a print log of each city as it's being processed (with both the city number and city name).

In [4]:
# Creating list to filled with the cities' data
cities_data = []

# Creates base of url
base = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&appid=" + api_key

# Begins both the record count and the set count at 1
rec_count = 1
set_count = 1

# Iterates through each city in the city list at a set limit and time delay
for i, city in enumerate(citylist):
    time.sleep(0.25)
    # After 50 cities, resets the record count to 0 and restarts the process.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        rec_count = 0
        
    # Prints City's Name along with its Record and Set Count
    print(f"Processing Record {rec_count} of set {set_count} | {city}")
    
    # Adds 1 to Record Count
    rec_count += 1
    url = f"{base}&q={city}"
    
    # Performs requests for API data to retrieve all variables needed
    try:
        weather_data = requests.get(url).json()
        city_tmp = weather_data['main']['temp']
        city_max_tmp = weather_data['main']['temp_max']
        city_humid = weather_data['main']['humidity']
        longitude = weather_data['coord']['lon']
        latitude = weather_data['coord']['lat']
        city_clouds = weather_data['clouds']['all']
        country = weather_data['sys']['country']
        city_wind_spd = weather_data['wind']['speed']
        date = weather_data['dt']
        
        # Add data requested into cities_data and convert to dictionary
        cities_data.append({"City" : city_tmp,
                           "Temperature" : city_max_tmp,
                           "Humidity" : city_humid,
                           "Longitude" : longitude,
                           "Latitude" : latitude,
                           "Cloudiness" : city_clouds,
                           "Country" :  country,
                           "Windspeed": city_wind_spd,
                           "Date": date
                           })
    # Adds exceptions when city is not found to keep the code running
    except:
        # Adds print statement for when exception is found
        print(f'City not found. Skipping...')
        pass

# Prints when its done running
print("-----------------------------")
print("Data Retrieval Complete")
print("-----------------------------")

Processing Record 1 of set 1 | ponta do sol
Processing Record 2 of set 1 | vaini
Processing Record 3 of set 1 | bud
Processing Record 4 of set 1 | port elizabeth
Processing Record 5 of set 1 | havoysund
Processing Record 6 of set 1 | khatanga
Processing Record 7 of set 1 | mys shmidta
City not found. Skipping...
Processing Record 8 of set 1 | bereda
Processing Record 9 of set 1 | tukrah
Processing Record 10 of set 1 | yellowknife
Processing Record 11 of set 1 | bonthe
Processing Record 12 of set 1 | busselton
Processing Record 13 of set 1 | tasiilaq
Processing Record 14 of set 1 | labrea
Processing Record 15 of set 1 | shingu
Processing Record 16 of set 1 | hilo
Processing Record 17 of set 1 | mar del plata
Processing Record 18 of set 1 | saint-joseph
Processing Record 19 of set 1 | kodiak
Processing Record 20 of set 1 | ushuaia
Processing Record 21 of set 1 | tripoli
Processing Record 22 of set 1 | tsihombe
City not found. Skipping...
Processing Record 23 of set 1 | lapeer
Processing 

Processing Record 40 of set 4 | tiksi
Processing Record 41 of set 4 | oranjemund
Processing Record 42 of set 4 | kemin
Processing Record 43 of set 4 | tvoroyri
Processing Record 44 of set 4 | burgeo
Processing Record 45 of set 4 | butaritari
Processing Record 46 of set 4 | georgetown
Processing Record 47 of set 4 | bundaberg
Processing Record 48 of set 4 | bafoulabe
Processing Record 49 of set 4 | ilulissat
Processing Record 0 of set 5 | oranjestad
Processing Record 1 of set 5 | lorengau
Processing Record 2 of set 5 | bambous virieux
Processing Record 3 of set 5 | clyde river
Processing Record 4 of set 5 | ostrovnoy
Processing Record 5 of set 5 | faanui
Processing Record 6 of set 5 | carolina
Processing Record 7 of set 5 | les cayes
Processing Record 8 of set 5 | valleyview
Processing Record 9 of set 5 | lagoa
Processing Record 10 of set 5 | paamiut
Processing Record 11 of set 5 | yumen
Processing Record 12 of set 5 | fortuna
Processing Record 13 of set 5 | bonavista
Processing Record 

Processing Record 29 of set 8 | dekoa
City not found. Skipping...
Processing Record 30 of set 8 | baghdad
Processing Record 31 of set 8 | mashhad
Processing Record 32 of set 8 | mahanoro
Processing Record 33 of set 8 | bac lieu
Processing Record 34 of set 8 | kachug
Processing Record 35 of set 8 | atar
Processing Record 36 of set 8 | lao cai
Processing Record 37 of set 8 | mackenzie
Processing Record 38 of set 8 | beloha
Processing Record 39 of set 8 | sindor
Processing Record 40 of set 8 | balykshi
Processing Record 41 of set 8 | tornio
Processing Record 42 of set 8 | sur
Processing Record 43 of set 8 | warri
Processing Record 44 of set 8 | barreirinha
Processing Record 45 of set 8 | sitka
Processing Record 46 of set 8 | anar darreh
City not found. Skipping...
Processing Record 47 of set 8 | carauari
Processing Record 48 of set 8 | saint-leu
Processing Record 49 of set 8 | farmington
Processing Record 0 of set 9 | seto
Processing Record 1 of set 9 | elizabethtown
Processing Record 2 o

Processing Record 16 of set 12 | inirida
Processing Record 17 of set 12 | cusuna
Processing Record 18 of set 12 | ngunguru
Processing Record 19 of set 12 | shache
Processing Record 20 of set 12 | san policarpo
Processing Record 21 of set 12 | iracoubo
Processing Record 22 of set 12 | batagay-alyta
Processing Record 23 of set 12 | cagayan de tawi-tawi
City not found. Skipping...
Processing Record 24 of set 12 | gimli
Processing Record 25 of set 12 | eyl
Processing Record 26 of set 12 | tilichiki
Processing Record 27 of set 12 | kodinsk
Processing Record 28 of set 12 | toliary
City not found. Skipping...
Processing Record 29 of set 12 | touros
Processing Record 30 of set 12 | huilong
Processing Record 31 of set 12 | pacific grove
Processing Record 32 of set 12 | mehamn
Processing Record 33 of set 12 | kibala
Processing Record 34 of set 12 | kayes
Processing Record 35 of set 12 | lolua
City not found. Skipping...
Processing Record 36 of set 12 | kutum
Processing Record 37 of set 12 | bala

###Convert Raw Data to DataFrame
* Exports the above city data into a .csv
* Displays the DataFrame

##Data Cleanup
* Inspects the data and removes any cities where the humidity > 100%
* Note, if there were no cities above that have humidity > 100%, this process is then skipped.

##Plotting the Data
Creates scatter plots that showcases the following relationships:
* Latitude vs. Temperature (F)
* Latitude vs. Humidity (%)
* Latitude vs. Cloudiness (%)
* Wind Speed (mph) vs. Latitude

###Latitude vs. Temperature Plot

###Latitude vs. Humidity Plot

###Latitude vs. Cloudiness Plot

###Latitude vs. Wind Speed Plot

##Linear Regression

###North Hemisphere - Max Temp (F) vs. Latitude Linear Regression

###South Hemisphere - Max Temp (F) vs. Latitude Linear Regression

###North Hemisphere - Humidity (%) vs. Latitude Linear Regression

###South Hemisphere - Humidity (%) vs. Latitude Linear Regression

###North Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

###South Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

###North Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

###South Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression